In [ ]:
import pandas as pd
import numpy as np
import sklearn as sklearn
import seaborn as sb
from PIL import Image
from matplotlib import image
from matplotlib import pyplot
import glob

from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier

from matplotlib import pyplot as plt



# Import Images from ORL dataset


In [ ]:
def get_all_images(path):
    image_list = []
    for x in range(1,41): #loop on every subject folder
        for filename in glob.glob(path + 's' + str(x) + '/*.pgm'): #get all images inside sX folder
            Im = Image.open(filename)
            image_list.append(Im)
    return image_list

# Data Setup


In [13]:
def generate_data_matrix(path):
  images = get_all_images(path)
  print(len(images)) #400

  #convert images list to a numpy array list
  images_vector = []
  for img in images:
      images_vector.append(np.array(img).ravel())

  print(len(images_vector)) #400
  print(images_vector[0].size) #10304
  print(images_vector[0].shape) #10304

  #df = pd.DataFrame(images_vector)
  y_labels = []
  for i in range(1,41):
      for j in range(1,11):
          y_labels.append(i)
  #df['label'] = pd.Series(y_labels)
  labels = np.asarray(y_labels , dtype = "int32")
  images_data = np.asarray(images_vector , dtype = "int32")
  return images_data , labels


0
0


IndexError: ignored

# The ORL dataset PATH


In [ ]:
images_data , images_label = generate_data_matrix('/Users/batsh/Desktop/ORL dataset/')

# Split the data for training and test sets


In [ ]:
# testset
data_matrix_Test = []
label_matrix_Test = []

#trainigset
data_matrix_Train = []
label_matrix_Train = []

for i,index in zip(range(400) , range(400)):
  if(i % 2 == 0):
    data_matrix_Test.append(images_data[index])
    label_matrix_Test.append(images_label[index])
  else:
    data_matrix_Train.append(images_data[index])
    label_matrix_Train.append(images_label)


# the trainig_dataset and test_dataset
Data_matix_test = np.asarray(data_matrix_Test , dtype = "int32")
label_matix_test = np.asarray(label_matrix_Test , dtype = "int32")

Data_matix_train = np.asarray(data_matrix_Train , dtype = "int32")
label_matix_train = np.asarray(label_matrix_Train , dtype = "int32")

# PCA

In [ ]:
##Implementaion of Princible component analysis
alpha = np.array(
    [0.8,0.85,0.9,0.95]
    )

def PCA():
  test_matrix_mean = np.mean(Data_matix_test , axis = 0)                                                    #compute mean of each dim. in the data matrix 
  train_matrix_mean = np.mean(Data_matix_train , axis = 0)

  test_Z =  Data_matix_test - test_matrix_mean
  train_Z = Data_matix_train - train_matrix_mean
                                                                     # center data step
  cov_train_matrix = np.cov(train_Z , rowvar = False)                                         # covariance matrix of centered data matrix Z
  eign_values , eign_vectors = np.linalg.eigh(cov_train_matrix)                         # compute the eign_val and eign_vectors of cov_matrix

  sorted_index = np.argsort(eign_values)[::-1]
  sorted_eign_values = eign_values[sorted_index]
  sorted_eign_vectors = eign_vectors[:sorted_index]

  sum_eign_values_d = np.linalg.det(sorted_eign_values)                           #compute the sum of eign_values in d dim.
  
  reduced_dimention = 10304
  #for each alpha value
  for i in alpha:
    
    for itr in range(1 , 10304):

      eign_values_r = sorted_eign_values[:,0:itr]                                   #subset of eign_values matrix to get the number of new dims.
      sum_eign_values_r = np.linalg.det(eign_values_r)                              #compute the sum of eign_Values in r dim matrix (subset)
      f_r = sum_eign_values_r / sum_eign_values_d                                   #f(r) = sum(r) / sum(d)
                                                                                    #check if the ratio is one of given alpha in pdf
    
      if(f_r >= i):
        reduced_dimention = itr                                                     #if any one is ture break
        break
  
    projection_matrix = sorted_eign_vectors[:,0:reduced_dimention]

    reduced_data_train_matrix = np.dot(projection_matrix.transpose() , train_Z.transpose()).transpose()
    reduced_data_test_matrix = np.dot(projection_matrix.transpose() , test_Z.transpose())

    accuracy_matrix = [] 
    #first neighbor
    KNN = KNeighborsClassifier(n_neighbors = 1)
    KNN.fit(reduced_data_train_matrix , label_matix_train)
    result = KNN.predict(reduced_data_test_matrix)
    accuracy_matrix.append(accuracy_score(result , label_matrix_Test))
    print("For First Neighbor")
    for k in range(len(accuracy_matrix)):
      print("Accuracy for alpha = " + str(i) + ": " + str(accuracy_matrix[k]))
    
    plt.plot(accuracy_matrix , alpha)
    plt.show()





